In [1]:
import yaml
import numpy as np
import matplotlib
import csv
import sys
import pandas as pd

from tkinter import filedialog
from PyQt5 import QtCore, QtWidgets
from PyQt5.QtCore import Qt
from matplotlib.figure import Figure
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar

from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QPushButton, QFileDialog

matplotlib.use('Qt5Agg')

In [2]:
def Gaussian_Dist(mean, stddev):
    return np.random.normal(loc=mean, scale=stddev)

def Beta_Dist(a, b):
    return np.random.beta(a, b)

def Lognormal_Dist(log_mean, sigma):
    return np.random.lognormal(mean=log_mean, sigma=sigma)

In [3]:
def computed_LP(mileage, SOCe, t0, Tf, Pm, E0, Pch0, eta, mode):
    LP = np.zeros(2880)

    Em = mileage * Pm
    Em_m = Em*60
    E0_m = E0*60000
    Tf_m = Tf*60

    Pch = Pch0*eta/100
    Pch_m = Pch*1000

    t0_m = t0*60

    if (mode == 1):                 # final SOC
        Ech = Em_m
        Tch = Ech/Pch_m
        np.ceil(Tch).astype(int)
        T = t0_m + Tch
        Tch = np.ceil(Tch)
        T = np.ceil(T)

    else:
        SOCi = (SOCe/100 - Em_m/E0_m)
        Ech = (1 - SOCi)*E0_m
        Tch = Ech/Pch_m
        if (Tf_m > Tch):
            T = t0_m + Tch
        else:
            T = t0_m + Tf
        Tch = np.ceil(Tch)
        T = np.ceil(T)

    LP[t0_m: int(np.ceil(T))] = Pch_m

    return LP, Tch, T

In [4]:
def get_customer_probability(customers_data, customer):
    if(customer == 'All' or customer == ''):
        customers_propability = np.array([])
        for cus in list(customers_data.keys())[1:]:
            customers_propability = np.append(customers_propability, customers_data[cus]['probability'])
        
    else:
        customers_propability = np.zeros(len(list(customers_data.keys())[1:]))
        idx = list(customers_data.keys())[1:].index(customer)
        customers_propability[idx] = 1
    
    customers_propability = customers_propability/np.sum(customers_propability)
    return customers_propability

In [5]:
def get_type_prob(customers_data, customer):
    type_propability = np.array([])
    for type in list(customers_data[customer].keys())[:-1]:
        type_propability = np.append(type_propability, customers_data[customer][type]['probability'])
    
    type_propability = type_propability/np.sum(type_propability)
    return type_propability

In [6]:
def compute_total_LP(all_distributions, customers_data, customer):
    Vehicle_database = pd.read_csv('data.csv')

    all_custormers = list(customers_data.keys())[1:]
    customers_propability = get_customer_probability(customers_data, customer)
    
    N = customers_data['Number_of_sample']

    # load profile initialization
    Load_profile = []
    Total_LP = np.zeros(2880)

    # # open the csv file for recording data of individual vehicle
    # with open('Individual_charging_data.csv', 'w', newline='\n') as cfile:
    # writer = csv.writer(cfile, delimiter=',')
    # elements = [" Customer Type ", " Electric Vehicle Type ", " mileage ", " SOC ", " Start Time of Charging "," Fixed Charging duration ", " Calculated Charging duration ", " End Time of Charging "]
    # writer.writerow(elements)

    for i in range(N):
        # Sample from customers
        cust = np.random.choice(all_custormers, 1, p=customers_propability)
        cust_data = customers_data[cust[0]]

        # discrete disttribution of customer's vehicle
        V = list(cust_data.keys())[0:-1]

        # probability = cust_data['Probability']
        probability = get_type_prob(customers_data, cust[0])

        Vehicle = np.random.choice(V, 1, p=probability)
        cust_vehicle_data = cust_data[Vehicle[0]]
        
        # read the distribution
        dist_mileage = cust_vehicle_data['mileage']
        dist_SOC     = cust_vehicle_data['SOC']
        dist_t0      = cust_vehicle_data['t0']
        dist_Tf      = cust_vehicle_data['Tf']
        mode         = cust_vehicle_data['mode']

        # sample from distribution        
        mileage = int(all_distributions[dist_mileage['Distribution']](**dist_mileage['Parameters']))
        SOC     = int(all_distributions[dist_SOC['Distribution']](**dist_SOC['Parameters']))
        t0      = int(all_distributions[dist_t0['Distribution']](**dist_t0['Parameters']))
        Tf      = int(all_distributions[dist_Tf['Distribution']](**dist_Tf['Parameters']))

        # vehicle parameters
        Pm = Vehicle_database.loc[Vehicle_database["Name"] == Vehicle[0], "Power consumption (Wh/km)"].values[0]
        E0 = Vehicle_database.loc[Vehicle_database["Name"] == Vehicle[0], "Battery capacity(kWh)"].values[0]
        Pch0 = Vehicle_database.loc[Vehicle_database["Name"] == Vehicle[0], "Charging power (kW)"].values[0]
        eta = 100

        # load profile calculation (single vehicle)
        Lp, Tch, T = computed_LP(mileage, SOC, t0, Tf, Pm, E0, Pch0, eta, mode)
        Load_profile.append(Lp)
        # csv_write_distribution = [cust[0], Vehicle[0], mileage, SOC, t0, Tf, Tch, T]

        # # write into csv:
        # writer.writerow(csv_write_distribution)

        # Total load profile accumulation
        Total_LP = np.add(Total_LP, Load_profile[i])
    # cfile.close()

    return Total_LP

In [7]:
# Read master customer data file YAML
def read_data(file_name):
    with open(file_name) as file:
        customers_data = yaml.safe_load(file)
    return customers_data

# some initialization
all_distributions = {'Gaussian': Gaussian_Dist,
                     'Beta': Beta_Dist, 
                     'Lognormal': Lognormal_Dist}

In [8]:
class Ui_MainWindow(object):

    def setupUi(self, MainWindow):
        
        # INTERFACE
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(1264, 869)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.verticalLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.verticalLayoutWidget.setGeometry(QtCore.QRect(10, 60, 1241, 721))
        self.verticalLayoutWidget.setObjectName("verticalLayoutWidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.verticalLayoutWidget)
        self.verticalLayout.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout.setObjectName("verticalLayout")
        self.Btn_Simulate = QtWidgets.QPushButton(self.centralwidget)
        self.Btn_Simulate.setGeometry(QtCore.QRect(670, 790, 111, 31))
        self.Btn_Simulate.setObjectName("Btn_Simulate")
        self.Btn_Export = QtWidgets.QPushButton(self.centralwidget)
        self.Btn_Export.setGeometry(QtCore.QRect(480, 790, 111, 31))
        self.Btn_Export.setObjectName("Btn_Export")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(640, 10, 71, 31))
        self.label.setObjectName("label")
        self.cBox_Customer = QtWidgets.QComboBox(self.centralwidget)
        self.cBox_Customer.setGeometry(QtCore.QRect(710, 10, 151, 31))
        self.cBox_Customer.setObjectName("cBox_Customer")
        self.Btn_Input = QtWidgets.QPushButton(self.centralwidget)
        self.Btn_Input.setGeometry(QtCore.QRect(370, 10, 91, 31))
        self.Btn_Input.setObjectName("Btn_Input")
        self.label_noti = QtWidgets.QLabel(self.centralwidget)
        self.label_noti.setGeometry(QtCore.QRect(350, 40, 131, 21))
        self.label_noti.setObjectName("label_noti")
        MainWindow.setCentralWidget(self.centralwidget)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

        # button
        self.Btn_Export.clicked.connect(self.export)
        self.Btn_Simulate.clicked.connect(self.simulate)
        self.Btn_Input.clicked.connect(self.select_input)

        self.label.setAlignment(Qt.AlignCenter)
        self.label_noti.setStyleSheet('color: red;')

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.Btn_Simulate.setText(_translate("MainWindow", "Simulate"))
        self.Btn_Export.setText(_translate("MainWindow", "Export"))
        self.label.setText(_translate("MainWindow", "Customer:"))
        self.Btn_Input.setText(_translate("MainWindow", "Input file"))
        self.label_noti.setText(_translate("MainWindow", "<html><head/><body><p align=\"center\"><span style=\" color:#ff0000;\"><br/></span></p></body></html>"))
        

    def select_input(self):
        filename, _ = QFileDialog.getOpenFileName(None, "Select input file...", "", "YAML Files (*.yml)")
        self.customers_data = read_data(filename)
        self.cBox_Customer.clear()
        self.cBox_Customer.addItem("All")
        self.cBox_Customer.addItems(list(self.customers_data.keys())[1:])
        self.label_noti.setText("Input file is loaded")


    def export(self):
        filename, _ = QFileDialog.getSaveFileName(filter="CSV Files (*.csv)")
        if filename:
            with open(filename, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(self.Total_LP)
        
    
    def simulate(self):
        # clear
        for i in reversed(range(self.verticalLayout.count())): 
            self.verticalLayout.itemAt(i).widget().setParent(None)

        # compute Total_LP and export csv
        customer = self.cBox_Customer.currentText()
        self.Total_LP = compute_total_LP(all_distributions, self.customers_data, customer)

        # plot
        sc = MplCanvas(self, width=5, height=4, dpi=90)
        x_value = np.arange(0, 2880)
        sc.axes.plot(x_value, self.Total_LP)

        # Create toolbar, passing canvas as first parament, parent (self, the MainWindow) as second.
        toolbar = NavigationToolbar(sc)
        self.verticalLayout.addWidget(toolbar)
        self.verticalLayout.addWidget(sc)


class MplCanvas(FigureCanvasQTAgg):
    def __init__(self, parent=None, width=5, height=4, dpi=200):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        self.axes.set_ylabel('Load Profile')
        self.axes.set_xlabel('Time')
        super(MplCanvas, self).__init__(fig)

In [9]:
if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.setWindowTitle("Load Profile Application")
    MainWindow.setFixedSize(1264, 869)
    MainWindow.setStyleSheet("QWidget {font-size: 11pt; }")

    MainWindow.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\tln229\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
